In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers
#from tensorflow.keras import Sequential
#from tensorflow.keras import Dense, Dropout
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, InputLayer, Reshape
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from keras import metrics

from keras.utils import np_utils

import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
import seaborn as sns

In [ ]:
# Helper to plot loss
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.legend()
  plt.grid(True)

# Keras and Tensorflow Optimizations

There are several things that we can do to make our networks a bit better. Unfortunately for much of this there aren't definitive answers for "what is the best choice", so we do have to do some trial and error, but we can use some guidelines to get us started in the right direction. 

## Load MNIST Data

We can use the MNIST digit dataset for testing, since it is reasonably large. 

In [ ]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

print(train_images.shape)

train_labels = np_utils.to_categorical(train_labels)
test_labels = np_utils.to_categorical(test_labels)

model = keras.Sequential()
model.add(InputLayer(input_shape=(28, 28)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(10, activation="softmax"))

# Train the digit classification model
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics='accuracy')

train_log = model.fit(
  train_images,
  train_labels,
  epochs=10,
  validation_split=0.2,
)
model.evaluate(test_images, test_labels)
plot_loss(train_log)

## Prequel - Saving and Loading Models

As we've seen, models can take a long time to train in many cases. Like with the sklearn models, we can save and load ours as they are trained and reused. This is a pretty integral part of making neural network models usable, so it is pretty easy. 

In addition to this we often see models saved in the h5 format, which just saves slightly less stuff along with the model. If we are using models trained elsewhere this format is very common. 

In [ ]:
# Save my model
model.save('model_path')
model = keras.models.load_model('model_path')

In [ ]:
# Calling `save('my_model.h5')` creates a h5 file `my_model.h5`.
model.save("my_h5_model.h5")

# It can be used to reconstruct the model identically.
reconstructed_model = keras.models.load_model("my_h5_model.h5")

## Network Size

Probably the first question that we will think of when building networks through Tensorflow is "how big should it be"? This is a very big question, and one of those ones without a real answer. We can put some guidelines in place to help us though. 

### What Does the Size Mean?

The size of a neural network is also known as the capacity. We can relate it roughly to the size of our first model, the tree. The larger a network is the higher its capacity to learn. This is similar to a tree, the larger the tree, the more fitted it can become to the training data. 

### What Size to Use?

We can start with a few guidelines to have a reasonably sized neural network. These steps do not ensure an optimal solution, but they'll get us started. There really is not a prescribed method for calculating the optimal network size (beleive me, I've looked), but there are several rules of thumb we can build together to get a rough estimate of a starting point for sizing:

<ul>
<li> Start with an input layer that is either
    <ul>
    <li> The width of the data, if the feature set is relatively small. 
    <li> A reasonably large number if the feature set is large. 
    <li> We don't have a true diving line, but 512 is a reasonable value to try for an upper end, at least at first. 
    </ul>
<li> Add 1 or 2 hidden layers of the same size and observe the results. We want to keep the model smaller if making it larger doesn't improve things, so first we shoudl see how good of a job a small model does. If the data is very large, skipping past the 1 layer step may save some time since we can predict that we can do better with a larger model in advance. 
<li> Increase layers of the same size until we get some overfitting and the training loss flattens. We want to reach the point where the model is getting to be excellent at predicting the training data. This is something we can see in the plot by noticing that the validation loss flatlines or starts to get worse. The training loss flattening is an indication that the model is not getting any better at learning the training data; we can use early stopping with a loose patience setting on training loss and lots of epochs to find this. 
<li> Add regularization steps to cut down that overfitting. We can try regularization and dropouts to cut down on that overfitting. We probably want to try a few options, parameters, and combinations here, there's not really a way to know in advance which regularization will work best on our data. 
<li> Do a test of "funneling" the layer size, potentially adding more layers. The traditional configuration of layers is to gradually decrease the size from the input layer towards the output layer. There is open debate on if this is better than having layers that are all the same size. We can play with this a little to see if results improve or not. 
<li> Use pruning. Much like a tree we can prune back a model to fight overfitting. 
</ul>

### Height vs Width

Another begged question is should we make networks wider (more neurons) or deeper (more layers)? To that effect, we can also think about what happens as layers are added. Each layer allows the network to learn a different representation of the data, and if we flash back to the simple logistic regression and XOR examples, each layer allows the model to capture relationships that are more complex:
<ul>
<li> No hidden layers - linear relationships only. 
<li> 1 or more hidden layers - nonlinear relationships, of increasing complexity.
</ul>

So the number of layers in our model directly relates to the complexity of relationships that we can capture. This doesn't directly mean that more layers are better, but it generally means that we want the number of layers to mirror the "complexity" of the data, though complexity is a term with no exact definition or metric here. Once again, there's no universal answer on the balance between width of layers and number of layers, but the general evidence leans towards more layers. There are several reasons for this, none of them definitive, but taken as a whole they add up to a strong case:

<ul>
<li> Ability to learn different representation of the data - this will be more clear next time when we start to look at some image specific neural networks, but one of the cool features of neural networks is that at each layer the network "sees" a different representation of the data, as it goes through each round of transformations. This has the effect of allowing it to identify different features at each layer, and use those features to make more and more accurate predictions. We'll examine this more soon. 
<li> Avoiding overfitting - extremely wide neural networks tend towards overfitting the training data and not generalizing as well to new data. 
<li> Ability to add interim steps - with a multi layer network we can add multiple steps such as regularization or dropouts, again to fight overfitting. 
<li> Automatic feature selection - deep neural networks will automatically perform a type of feature selection as the least important features are minimized in their importance. This is an emerging area of research - some people have argued that well designed neural networks can remove the need for feature selection, and neural networks are being created to be feature selection tools. We can see this illustrated most clearly with images again, we feed a network an entire image, and get a prediction. Note that this isn't a total rejection of feature selection for neural networks, improving the feature set will impact neural network models just as it will for ordinary models; with neural networks we just have the potential for the network to "cover for mistakes" in the features. This is more dramatic as data size and network size increase. 
<li> Results - deep learning has become a common term recently for a reason, due to the success of deep neural networks with many layers. Most of the cool stuff that we see coming from AI such as image recognition, translation, and self navigating robots are the result of deep learning networks. In practice these networks have tended to outperform shallower ones, especially in more complex tasks. 
</ul>

### Overfitting and Underfitting

Why not make a model that is both very wide and very deep? This will tend to overfit as it can "memorize" the training data. With large datasets we do see very large models in some cases, since the more data we have, the more fitting we can handle. With large datasets and huge models, the training time can potentially explode, so we have to be careful. This is similar to what we saw with unlimited size in trees, we can eventually create a model large enough to memorize the training data. On the whole, the model capacity can be thought of similarly to the size of a tree. If we have large amounts of complex data, we want a model that has a very high capacity, as the relationships are complicated and we have enough data to mitigate overfitting. If we have smaller or more simple data, we want a smaller model, as a large one will overfit our data. 

Finding the "optimal" size is still somewhat of an art, combined with patience for trial and error. <b>The easiest starting point is probably to create a model that can potentially overfit, though not comically large, then use some regularization techniques and early stopping to find the optimal fit.</b> We should also note that it is possible and reasonable that models of different sizes and configurations tend to converge on similar levels of performace. These neural networks are extremely flexible in how they learn during the training process, so they can "learn around" some degree of design decisions in a way that is more dramatic than our previous models. 

In [ ]:
# Model
model = keras.Sequential()
model.add(InputLayer(input_shape=(28, 28)))
model.add(Flatten())
model.add(Dense(784, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(784, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation="softmax"))

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True) 

# Train the digit classification model
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics='accuracy')

train_log = model.fit(
  train_images,
  train_labels,
  epochs=100,
  validation_split=0.2,
  callbacks=[callback]
)
model.evaluate(test_images, test_labels)
plot_loss(train_log)

In [ ]:
# Bigger Model
model = keras.Sequential()
model.add(InputLayer(input_shape=(28, 28)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation="softmax"))

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True) 

# Train the digit classification model
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics='accuracy')

train_log = model.fit(
  train_images,
  train_labels,
  epochs=100,
  validation_split=0.2,
  callbacks=[callback]
)
model.evaluate(test_images, test_labels)
plot_loss(train_log)

In [ ]:
# Tapered Model
model = keras.Sequential()
model.add(InputLayer(input_shape=(28, 28)))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(350, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation="softmax"))

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True) 

# Train the digit classification model
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics='accuracy')

train_log = model.fit(
  train_images,
  train_labels,
  epochs=100,
  validation_split=0.2,
  callbacks=[callback]
)
model.evaluate(test_images, test_labels)
plot_loss(train_log)

## Epochs and Batch Sizes

![Batch Sizes](images/iterations_epoch.webp "Batch Sizes")

### Epochs

Each epoch is a run through all of the training data. Epochs are simple, we can set a large number and use early stopping to cut things off when we've reached the best result. 

### Batch Sizes

Batch size determines how many records are processed before the gradients are updated - i.e. the number of records between one forward and backwards pass. The batch sizes are a matter of very open debate for the optimal solution. At the high end, batch sizes are limited by what can fit in memory. When dealing with very large data this may matter as a batch that is a small fraction of the data may be a massive absolute size. At the lower end using smaller batches gives the same effect as it does when we looked at regular gradient descent - the gradients become less stable as we are relying on a smaller number of records. In reading more about batch sizes I want to update my recommendation to be even smaller than the 50 to 150 I suggested before, down to less than 100, even as small as into the single digits. There is research that smaller batch sizes tend to produce models that generalize better than ones with larger batches. 

Larger batch sizes do tend to be processed more quickly, sometimes substantially so, as the hardware is better able to be "saturated" with data to process. In big data scenarios, this can matter. One thing that you see in practice is that the GPUs (or similar) that are used to train neural networks have a certain amount of memory, and the batch size is limited by that memory. When doing something that involves large images or video, or similar, this can be a real area for concern. For us, these constraints won't really come up, but it's good to be aware of them. 

Dont' stress too much on batch size, this is really something that needs to be grid searched to find a great answer and in scenarios where it actually matters, is largely influenced by the hardware that is available.

In [ ]:
# Big Batch
model = keras.Sequential()
model.add(InputLayer(input_shape=(28, 28)))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(350, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation="softmax"))

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True) 

# Train the digit classification model
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics='accuracy')

train_log = model.fit(
  train_images,
  train_labels,
  epochs=100,
  batch_size=5000,
  validation_split=0.2,
  callbacks=[callback]
)
model.evaluate(test_images, test_labels)
plot_loss(train_log)

In [ ]:
# Small Batch
model = keras.Sequential()
model.add(InputLayer(input_shape=(28, 28)))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(350, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation="softmax"))

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True) 

# Train the digit classification model
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics='accuracy')

# Note: Batch size of 2 is good to test
# But it can take a VERY long time, be aggressive with the early stopping
train_log = model.fit(
  train_images,
  train_labels,
  epochs=300,
  batch_size=2,
  validation_split=0.2,
  callbacks=[callback]
)
model.evaluate(test_images, test_labels)
plot_loss(train_log)

## Optimizer

Of all options the optimizer is the one we will care about the least. Each different optimizer is a different algorithm for doing the gradient descent. The optimizers have different results with respect to speed, memory usage, computational expense, and likelyhood to get stuck in a local minima. The optimizer is similar to some of the options we saw with logistic regression - there were several solver options for that model, each of which performed the gradient descent process with a slightly different set of calculations. Optimizers tend to manipulate the learning rate, attempting to narrow in on the optimum weights in fewer epochs and thus fewer calculations.

![Optimizers](images/optimizers.gif "Optimizers")

<b>Note:</b> this animation doesn't have Adam, which is unfortumate, but it was the most clear one I could find. The different optimizers are all trying to find the same minimum, and in most cases they all do, but they take differnt paths to get there, over a different number of steps.

Adam is a good compromise between all factors and is very commonly used. We'll just use this for our work. One other common one is RMSprop, if you're feeling spicy, give that a try and see if there are any imporvements. These optimizers don't change the model we are making (outside of edge cases, like getting stuck in a minima), they change the process of finding that model. The biggest impact of the optimizer is on the speed of the training process, and in turn the ability to experiment with different models. This is more of a concern as the data gets larger, as small improvements on each individual gradient descent step can add up to large improvements in the overall training time. 

In [ ]:
optimizer_1 = tf.keras.optimizers.Adam()
optimizer_2 = tf.keras.optimizers.RMSprop()

## Activation 

Activation functions are the key to adding non-linearity to the network allowing it to learn complex and non-linear relationships in the data. We've used ReLU as the default and that is a solid choice in most cases. ReLU has one issue, the dying ReLU problem. This can happen when we get inputs to the activation function fall in the negative area. In short there can be neurons that "die" and never get updated again because the value becomes 0 and stays 0. These dead neurons are a problem as they now aren't contributing to the learning.

![ReLU](images/relu.jpeg "ReLU")

To combat the dying ReLU problem there are a couple of other activation functions that avoid that issue - Leaky ReLU and ELU. Each one changes the negative values to something other than 0 - Leaky ReLU uses a slight linear gradient, ELU uses an exponential function for a similar, but curved, slight gradient. These ReLU variants are a good choice, and probably the 'best' overall activation functions for most scenarios. One some datasets will be impacted by the dying ReLU problem. 

These activation function also have an impact on the speed of training. The ReLU function is very fast to calculate, and the Leaky ReLU and ELU functions are a bit slower. Other activation functions may be even more expensive. Will this matter? As with many things, it depends. For the small examples we are using, it probably won't matter much. On very large applications, we may need to consider the ability to train models more quickly, and try more models vs. the improved fit of another activation function.

#### Activation Function Guidelines

We can write a few rules of thumb to guide us in deciding on activation functions. On the whole, the choice is like a hyperparameter choice, and we want to choose whichever is the best for our data. Some of the guidelines are:
<ul>
<li> The output layer should have an activation function that matches the type of problem we are solving. 
    <ul>
    <li> <b>Regression:</b> Linear activation function.
    <li> <b>Binary Classification:</b> Sigmoid activation function.
    <li> <b>Multiclass Classification:</b> Softmax activation function.
    </ul>
<li> Depending on the type of network/problem, our hidden layers default to different activation functions:
    <ul>
    <li> <b>Deep Neural Networks:</b> ReLU activation functions.
    <li> <b>Convolutional Neural Networks: (Images)</b> ReLU activation functions.
    <li> <b>Recurrent Neural Networks:(Seqential, Time Series)</b> Tanh activation function.
    </ul>
</ul>

There are more activation functions, and others are being developed somewhat regularly. If in doubt, just use a ReLU variant. 

In [ ]:
# Take a leak 
model = keras.Sequential()
model.add(InputLayer(input_shape=(28, 28)))
model.add(Flatten())
model.add(Dense(500, activation='leaky_relu'))
model.add(Dropout(0.2))
model.add(Dense(350, activation='leaky_relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation="softmax"))

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True) 

# Train the digit classification model
model.compile(optimizer=optimizer_2, loss="categorical_crossentropy", metrics='accuracy')

train_log = model.fit(
  train_images,
  train_labels,
  epochs=10,
  batch_size=64,
  validation_split=0.2,
  callbacks=[callback]
)
model.evaluate(test_images, test_labels)
plot_loss(train_log)

## Initialization

The initialization provides the starting point for all the weights and bias values that we start out with. We initially started with random values in the scratch network - this is generally fine, but we can sometimes do better. One specific case where we can do better is when we have data that is highly imbalanced. This is a common problem in things such as fraud detection, where we have a very small number of fraud cases, and a very large number of non-fraud cases. Seeding the model with an "expectation" of the bias values can help the model learn faster and converge on an answer more quickly. In cases where we have a lot of data, this can be a big deal. In cases where we may have local minima in the loss curve, this can be significant - the more "ground" the gradient descent covers, the greater the odds it encounters a local minima that might be a trap. With imbalenced data, we expect that the model will need to cover a lot of that ground, so starting off with a better bias value can help. 

### Imbalanced Weighting

One application where initialization can help significantly is when dealing with imbalanced data. In this example of credit card fraud (real data that has been put through PCA), very, very few transactions are fraudulent. So we have a very imbalanced target value - the class variable. 

In [ ]:
file = tf.keras.utils
raw_df = pd.read_csv('https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv')
raw_df.head()

### Count the Target Outcomes

Credit card fraud is relatively rare, at least in view of the total number of transactions. We can count up the target values to see exactly what the expected skew is. 

In [ ]:
# Bincount will count the number in each category
neg, pos = np.bincount(raw_df['Class'])
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

### We Have an Imbalance

A big one. 

In [ ]:
# pop just removes a column. The equivalent of how we normally drop. 
# the TF docs commonly use this, so I've left it as is. 
cleaned_df = raw_df.copy()
# You don't want the `Time` column.
cleaned_df.pop('Time')

In [ ]:
# Use a utility from sklearn to split and shuffle your dataset.
train_df, test_df = train_test_split(cleaned_df, test_size=0.2)

# Form np arrays of labels and features.
train_labels = np.array(train_df.pop('Class'))
test_labels = np.array(test_df.pop('Class'))

train_features = np.array(train_df)
test_features = np.array(test_df)

In [ ]:
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)

#train_features = np.clip(train_features, -5, 5)
#test_features = np.clip(test_features, -5, 5)

print('Training features shape:', train_features.shape)
print('Training labels shape:', train_labels.shape)
print('Test features shape:', test_features.shape)
print('Test labels shape:', test_labels.shape)

### Create a Biased Model

The bias of the data is inserted in the model compilation step on the output layer. What does this do? It preconfigures the output layer to "expect" results to be this skewed. Recall that, along with the weight, the bias values are one of the things that is learned in training. By default the initial values are randomized, so the model needs to learn the skew towards the imbalance - if the balance between classes is moderate, that's not a big deal; if the balance is so drastically skewed in one direction, that's less practical. With the preset bias we can speed convergance and likely reduce loss. 

#### Other Imbalenced Work

Other things that we looked at to improve balance such as under/over sampling still works with neural networks as it would with anything else. This is just one nn-specific thing that we can implement with minimal extra work. 

#### Metrics

We can also add a bunch of metrics to what we get returned by creating a list of the metrics that we want. Keras.metrics has a list, they are all the metrics we might expect. 

In [ ]:
METRICS = [
      #keras.metrics.TruePositives(name='tp'),
      #keras.metrics.FalsePositives(name='fp'),
      #keras.metrics.TrueNegatives(name='tn'),
      #keras.metrics.FalseNegatives(name='fn'), 
      #keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

initial_bias = np.log([pos/neg])

output_bias = tf.keras.initializers.Constant(initial_bias)
model = keras.Sequential()
model.add(keras.layers.Dense(16, activation='relu',input_shape=(train_features.shape[-1],)))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias))
model.summary()

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True) 

#model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),loss=keras.losses.BinaryCrossentropy(),metrics=metrics)
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=METRICS)
#Fit
train_log = model.fit(
  train_features,
  train_labels,
  epochs=30,
  batch_size=64,
  validation_split=0.2,
  callbacks=[callback]
)
model.evaluate(test_features, test_labels)
plot_loss(train_log)

## Pruning

We can also use pruning to improve our networks, which is built into Tensorflow and is similar in concept to the tree pruning we did earlier. 

### Pruning Results

Pruning removes the least useful weights, increasing sparsity. These sparse models require less processing (since many calculations will be M * 0) and may be compressed down to take less space in memory. 

### Smaller Models

One other consideration is that we can use pruning to create smaller models that are better able to be executed on weaker hardware. In the context of a full computer, creating a prediction with a neural network is pretty fast. If we want to move the model to small embedded devices though, the memory and processing needs can still be excessive. Some scenarios where this comes up are things like security cameras that can recognize images, robots that can navigate themselves, or evern small computers like a Raspberry Pi. For example, if you have a self driving small car, you may collect training data from the car's camera and sensors (throttle, steering, etc..), train a model that can produce a "how to drive" prediction on a computer, then export a condensed version of that model that can run on the car's smaller low powered processor to make the predictions as the car drives. This challenge is magnified if you are dealing with something like video, which can generate 30+ images per second. Small models that are almost as good, but can be run with less compouting power allow the power of neural networks to be expanded to more devices - train on a powerful computer, us on a small and weak computer.

We can use the tflite set of tools to create special models that are optimized for lower computing power devices, though we won't explore that here. 

**Note:** pruning is largely a step that is for deployment of models, as we can make the processing more efficient and the memory required lesser. For us, it isn't the most critical of steps. In general, a smaller model that produces the same or similar accuracy is better, as you can do more with less. The reduction of overfitting and potential accuracy benefits are somewhat secondary to making the model more usable in practice. 

In [ ]:
!pip install tensorflow_model_optimization

In [ ]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
model_for_pruning = prune_low_magnitude(model)


model_for_pruning.compile(optimizer="adam",loss="binary_crossentropy",metrics=METRICS)

model_for_pruning.summary()

In [ ]:
# Add Callbacks
callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
]

train_log = model_for_pruning.fit(
                                  train_features,
                                  train_labels,
                                  batch_size=64,
                                  epochs=20,
                                  validation_split=.2,
                                  callbacks=callbacks
                                  )
print(model_for_pruning.evaluate(test_features, test_labels))
plot_loss(train_log)

## Exercise

Predict the price of diamonds! (The example solutions are fairly extreme in terms of the approach they take, you'll probably be ok with less dramatic approaches.)

![Diamonds](images/diamonds.jpeg "Diamonds" )

In [ ]:
ex_df = sns.load_dataset("diamonds")
ex_df = pd.get_dummies(ex_df)
ex_df.head()

In [ ]:
y = ex_df["price"]
X = ex_df.drop(columns={"price"})
X_tr_ex, X_te_ex, y_tr_ex, y_te_ex = train_test_split(X, y)
start_width = X.shape[1]
start_width

#### Generate a Baseline

I'll use a different loss - mean absolute percentage. 

In [ ]:
#baseline
# Make a model to set a baseline of performance

### Attempt Optimization

Looks like lots of loss! What to do?

Things to try:
<ol>
<li> Depth.
<li> Width. 
<li> Activations. 
<li> Batches. 
</ol>

In [ ]:
#Comically Deep Model


##### Try Adding Dropouts

We don't have a bunch of overfitting, so we may not expect miracles here... 

In [ ]:
#Comically Deep Model with Dropouts


##### Taper Model Somewhat

In [ ]:
#Taper Deep Model
